In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import LinuxImage

img = LinuxImage.from_str("6.5.0-27-generic-amd64")

img.section_info

,type,flags,offset,size,link,info,addralign,entsize,addr,data
name,,,,,,,,,,
,SHT_NULL,0,0,0,0,0,0,0,0,b''
.text,SHT_PROGBITS,6,2097152,20971520,0,0,4096,0,18446744071578845184,b'\xfc\x0f\x01\x15\x80\xc4u\x03\xb8\x10\x00\x0...
.rodata,SHT_PROGBITS,3,23068672,7227958,0,0,4096,0,18446744071599816704,b'initcall_blackl'
.pci_fixup,SHT_PROGBITS,2,30296640,14624,0,0,16,0,18446744071607044672,b'\x86\x80\xc3\x0e\xff\xff\xff\xff\x00\x00\x00...
.tracedata,SHT_PROGBITS,2,30311264,120,0,0,1,0,18446744071607059296,b'\x88\x03\x0c\xcf\xa1\x82\xff\xff\xff\xff\xd0...
__ksymtab,SHT_PROGBITS,2,30311384,69504,0,0,4,0,18446744071607059416,b'\xc8\xea\xe6\x01J\xfe\x05\x00\xae\xa1\x03\x0...
__ksymtab_gpl,SHT_PROGBITS,2,30380888,83664,0,0,4,0,18446744071607128920,b'x\xba\xda\xfe\x06k\x03\x00.\x92\x02\x00\x9c\...
__kcrctab,SHT_PROGBITS,2,30464552,23168,0,0,4,0,18446744071607212584,"b':y\x13t\xa3""\xb9\x1eg\x88v]\xb9nA'"
__kcrctab_gpl,SHT_PROGBITS,2,30487720,27888,0,0,4,0,18446744071607235752,b'\xd7\xb9\x8e\xe5\xf69Q\xb5\x1a\xe0\xa4 37\x97'


In [3]:
start_ftrace_events = img.get_value_by_name("__start_ftrace_events")
stop_ftrace_events = img.get_value_by_name("__stop_ftrace_events")

In [4]:
from depsurf import Kind

btf = img.btf

t = btf.get(Kind.STRUCT, "trace_event_call")


def get_offset_map(t):
    return {m["name"]: m["bits_offset"] for m in t["members"]}


get_offset_map(t)

class_offset = get_offset_map(t)["class"] // 8
print_fmt_offset = get_offset_map(t)["print_fmt"] // 8
tp_offset = get_offset_map(t)["tp"] // 8
flags_offset = get_offset_map(t)["flags"] // 8


[      btf.py:15 ] INFO: Loading /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/6.5.0-27-generic-amd64.pkl


In [5]:
t = btf.get(Kind.STRUCT, "trace_event_class")

probe_offset = get_offset_map(t)["probe"] // 8
system_name_offset = get_offset_map(t)["system"] // 8

In [6]:
t = btf.get(Kind.STRUCT, "tracepoint")

name_offset = get_offset_map(t)["name"] // 8

In [27]:
TRACE_EVENT_FL_TRACEPOINT = {e['name']: e['val'] for e in btf.get(Kind.ENUM, "(anon)")["values"]}["TRACE_EVENT_FL_TRACEPOINT"]

In [32]:
import logging


def handle_tp(trace_event_call):
    flags = img.get_int32(trace_event_call + flags_offset)

    if not (flags & TRACE_EVENT_FL_TRACEPOINT):
        return

    tp_ptr = img.get_int64(trace_event_call + tp_offset)
    tp = img.get_int64(tp_ptr)
    name = img.get_cstr(tp + name_offset)

    for func_prefix in ["perf_trace_", "__traceiter_", "btf_trace_"]:
        func = btf.get(Kind.FUNC, f"{func_prefix}{name}")
        if func:
            break
    else:
        raise ValueError(f"Could not find function for {name}")
    
    trace_event_class = img.get_int64(trace_event_call + class_offset)
    probe = img.get_int64(trace_event_class + probe_offset)
    cls_name = img.get_name_by_value(probe).removeprefix("trace_event_raw_event_")
    event_name = f"trace_event_raw_{cls_name}"

    event_struct = btf.get(Kind.STRUCT, event_name)
    print(name)
    print(f"\tFunc: {func}")
    print(f"\tEvent: {event_struct}")
    print()


for ptr in range(start_ftrace_events, stop_ftrace_events, 8):
    handle_tp(img.get_int64(ptr))

initcall_finish
	Func: {'kind': 'FUNC', 'name': 'perf_trace_initcall_finish', 'type': {'kind': 'FUNC_PROTO', 'params': [{'name': '__data', 'type': {'kind': 'PTR', 'type': {'name': 'void', 'kind': 'VOID'}}}, {'name': 'func', 'type': {'kind': 'TYPEDEF', 'name': 'initcall_t'}}, {'name': 'ret', 'type': {'kind': 'INT', 'name': 'int'}}], 'ret_type': {'name': 'void', 'kind': 'VOID'}}}
	Event: {'kind': 'STRUCT', 'name': 'trace_event_raw_initcall_finish', 'size': 24, 'members': [{'name': 'ent', 'bits_offset': 0, 'type': {'kind': 'STRUCT', 'name': 'trace_entry'}}, {'name': 'func', 'bits_offset': 64, 'type': {'kind': 'TYPEDEF', 'name': 'initcall_t'}}, {'name': 'ret', 'bits_offset': 128, 'type': {'kind': 'INT', 'name': 'int'}}, {'name': '__data', 'bits_offset': 160, 'type': {'kind': 'ARRAY', 'name': '(anon)', 'nr_elems': 0, 'type': {'kind': 'INT', 'name': 'char'}}}]}

initcall_start
	Func: {'kind': 'FUNC', 'name': 'perf_trace_initcall_start', 'type': {'kind': 'FUNC_PROTO', 'params': [{'name': '__d